Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
import pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [19]:
batch_size = 128
num_hidden_layer = 1024

graph = tf.Graph()
with graph.as_default():
    
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32,shape = (batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape = (batch_size,num_labels))
    tf_lambda = tf.placeholder(tf.float32)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    
    #weights and biases
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size,num_hidden_layer]))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer]))
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer,num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    #set nureal nerwork graph
    lay1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
    logits = tf.matmul(lay1,weights2)+biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels = tf_train_labels,logits = logits))+tf_lambda*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predict
    train_prediction = tf.nn.softmax(logits)
    
    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1) 
    valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
    
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)                                   

In [25]:
#经过测试，tf_lambda取0.01是较好的
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_lambda:0.01}
        _,l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if(step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3453.440430
Minibatch accuracy: 14.1%
Validation accuracy: 24.6%
Minibatch loss at step 500: 21.226551
Minibatch accuracy: 91.4%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.953851
Minibatch accuracy: 81.2%
Validation accuracy: 83.1%
Minibatch loss at step 1500: 0.587096
Minibatch accuracy: 86.7%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.605412
Minibatch accuracy: 89.8%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 0.712943
Minibatch accuracy: 84.4%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.762723
Minibatch accuracy: 82.0%
Validation accuracy: 83.4%
Test accuracy: 90.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [33]:
batch_size = 128
num_hidden_layer = 1024

graph = tf.Graph()
with graph.as_default():
    
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32,shape = (batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape = (batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    
    #weights and biases
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size,num_hidden_layer]))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer]))
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer,num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    #set nureal nerwork graph
    lay1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
    
    logits = tf.matmul(lay1,weights2)+biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels = tf_train_labels,logits = logits))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predict
    train_prediction = tf.nn.softmax(logits)
    
    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1) 
    valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
    
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)   

In [30]:
num_steps = 100

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _,l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if(step % 10 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 400.123016
Minibatch accuracy: 7.0%
Validation accuracy: 25.4%
Minibatch loss at step 10: 122.685043
Minibatch accuracy: 74.2%
Validation accuracy: 73.4%
Minibatch loss at step 20: 74.297531
Minibatch accuracy: 78.9%
Validation accuracy: 74.0%
Minibatch loss at step 30: 53.068359
Minibatch accuracy: 82.0%
Validation accuracy: 77.1%
Minibatch loss at step 40: 115.690338
Minibatch accuracy: 68.8%
Validation accuracy: 70.0%
Minibatch loss at step 50: 39.003601
Minibatch accuracy: 79.7%
Validation accuracy: 77.3%
Minibatch loss at step 60: 45.368530
Minibatch accuracy: 73.4%
Validation accuracy: 76.0%
Minibatch loss at step 70: 54.570244
Minibatch accuracy: 72.7%
Validation accuracy: 74.3%
Minibatch loss at step 80: 55.599964
Minibatch accuracy: 72.7%
Validation accuracy: 71.7%
Minibatch loss at step 90: 21.991791
Minibatch accuracy: 82.0%
Validation accuracy: 77.5%
Test accuracy: 85.8%


只是改了num_steps，去掉了惩罚项，确实出现了一定的过拟合

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [34]:

batch_size = 128
num_hidden_layer = 1024

graph = tf.Graph()
with graph.as_default():
    
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32,shape = (batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape = (batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    
    #weights and biases
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size,num_hidden_layer]))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer]))
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer,num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    #set nureal nerwork graph
    lay1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
    drop = tf.nn.dropout(lay1,0.5)
    logits = tf.matmul(drop,weights2)+biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels = tf_train_labels,logits = logits))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predict
    train_prediction = tf.nn.softmax(logits)
    
    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1) 
    valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
    
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2) 

In [35]:
num_steps = 100

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _,l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if(step % 10 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 597.078125
Minibatch accuracy: 9.4%
Validation accuracy: 32.2%
Minibatch loss at step 10: 132.847717
Minibatch accuracy: 74.2%
Validation accuracy: 72.1%
Minibatch loss at step 20: 100.311165
Minibatch accuracy: 73.4%
Validation accuracy: 75.7%
Minibatch loss at step 30: 67.605492
Minibatch accuracy: 76.6%
Validation accuracy: 77.3%
Minibatch loss at step 40: 161.917480
Minibatch accuracy: 70.3%
Validation accuracy: 73.1%
Minibatch loss at step 50: 79.408943
Minibatch accuracy: 72.7%
Validation accuracy: 77.9%
Minibatch loss at step 60: 111.958572
Minibatch accuracy: 64.1%
Validation accuracy: 74.3%
Minibatch loss at step 70: 91.117271
Minibatch accuracy: 64.8%
Validation accuracy: 74.5%
Minibatch loss at step 80: 79.934883
Minibatch accuracy: 70.3%
Validation accuracy: 78.7%
Minibatch loss at step 90: 44.646858
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Test accuracy: 87.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
